# Numerical Methods for Manu Body Physics, Assignment #4

Yoav Zack, ID 211677398

In [ ]:
using LinearAlgebra
using Statistics
using Plots
using Printf
using LsqFit
theme(:default)

## Question 1: 2D XY model and the Kosterlitz-Thouless transition

### 1. Wolff Algorithm

We want to implement a Monte-Carlo algorithm using Wolff Clusters for the 2D XY Model. In order to do so, we need the following functions:
1. `WolffUpdate()` - Updates a grid of spins 
2. `selectCluster()` - Selects a cluster according to Wolff method
3. `nearestNeighboors()` - Returns a list of nearest neighboors of a given site assuming given boundary conditions.

It will be easier for us to work if some of the data will be stores ina a `Grid` structure, as follows:

In [ ]:
struct Grid
    N::Integer
    grid::Matrix{Real}
    J::Real
    beta::Real
    bc::String
    
    function Grid(N::Integer, J::Real, T::Real, bc::String)
        @assert N > 0 "Grid size must be positive integer"
        @assert J > 0 "Interaction coefficient J must be positive"
        @assert T > 0 "Temperature must be positive"

        if !(bc in ["periodic", "closed"])
            error("Boundacy conditions are either periodic or closed")
        end
        
        beta = 1. / T
        grid = (2*pi) .* rand(N,N)
        new(N, grid, J, beta, bc)
    end
end

We will also define a simple function which plots the grid data ina a heatmap:

In [ ]:
function showGrid(grid::Matrix, T::Real)
    N = size(grid)[1]
    h = heatmap(grid, c=:cyclic_mygbm_30_95_c78_n256_s25, colorbar=nothing)
    heatmap!(h, xlim=(1,N), ylim=(1,N), xticks=0, yticks=0)
    heatmap!(h, aspect_ratio=:equal, frame=:on)
    heatmap!(h, title=@sprintf("N=%d, T=%.2f", N, T))
    return h
end

Now, we can start working on the functions mentioned above. First, the `nearestNeighboors()` function:

In [ ]:
function nearestNeighboors(i::Integer, j::Integer, grid::Grid, dir::String)
    N = grid.N
    if grid.bc == "periodic"
        i = (i-1 % N) + 1
        j = (j-1 % N) + 1
    end

    @assert i >= 1 && j >= 1 && i <= N && j <= N "Invalid spin index"
    neighboors = [(i+1,j), (i-1,j), (i,j+1), (i,j-1)]

    @assert dir in ["x", "y", "all"] "Invalid spin current direction " * dir
    if dir == "x"
        neighboors = neighboors[1:2]
    elseif dir == "y"
        neighboors = neighboors[3:4]
    end
    
    if grid.bc == "periodic"
        return [mod.(a .- 1, N) .+ 1 for a in neighboors]
    elseif grid.bc == "closed"
        return [(x,y) for (x,y) in neighboors if (x>=1) && (y>=1) && (x<=N) && (y<=N)]
    else
        error("Invalid boundary condition in nearestNeighboors()")
    end
end

Notice that we solved it for both periodic and closed boundary conditions. Next, we work on the Wolff part - selecting a cluster. For it (and the next section), we will use the following identities:
$$
\begin{array}{rl}
\vec{S}_{i} & =\left(\cos\theta_{i},\sin\theta_{i}\right)\\
\hat{e} & =\left(\cos\theta_{e},\sin\theta_{e}\right)\\
\vec{S}_{i}\cdot\hat{e} & =\cos\theta_{i}\cos\theta_{e}+\sin\theta_{i}\sin\theta_{e}=\cos\left(\theta_{i}-\theta_{e}\right)\\
\vec{S}_{i}-2\left(\vec{S}_{i}\cdot\hat{e}\right)\hat{e} & =\left(\cos\theta_{i},\sin\theta_{i}\right)-2\cos\left(\theta_{i}-\theta_{e}\right)\left(\cos\theta_{e},\sin\theta_{e}\right)\\
 & =\left(-\cos\left(\theta_{i}-2\theta_{e}\right),\sin\left(\theta_{i}-2\theta_{e}\right)\right)\\
 & =\left(\cos\left(\pi-\theta_{i}+2\theta_{e}\right),\sin\left(\pi-\theta_{i}+2\theta_{e}\right)\right)
\end{array}
$$
Thus the function for selecting a cluster is:

In [ ]:
function selectCluster(thetae::Real, i::Integer, j::Integer, grid::Grid)
    cluster = [(i,j)]
    queue = nearestNeighboors(i, j, grid, "all")
    
    thetai = grid.grid[i,j]
    prodi = cos(thetai - thetae)
    while !isempty(queue)
        s = pop!(queue)
        if s in cluster
            continue
        end
        thetaj = grid.grid[s[1], s[2]]
        prodj = cos(thetaj - thetae)
        p = 1 - exp(min(0, -2*grid.beta*grid.J*prodi*prodj))
        
        if rand() <= p
            push!(cluster, s)
            append!(queue, nearestNeighboors(s[1], s[2], grid, "all"))
        end
    end

    return cluster
end

This gives us a list of indices to by sliced by. Next, we write a function which does a single Wolff spin flip according to the function given in the assignment and the previously done calculations:

In [ ]:
function iterationWolff!(grid::Grid)
    i = rand(1:grid.N)
    j = rand(1:grid.N)

    thetae = (2*pi)*rand()
    cluster = selectCluster(thetae, i, j, grid)
    
    for s in cluster
        grid.grid[s[1], s[2]] = mod(pi - grid.grid[s[1], s[2]] + 2*thetae, 2*pi)
    end
end

### 2. Autocorrelation Time

Now we want to verify whether the autocorrelation and equlibration time for various system sizes and temperatures. We define a function to perform a complete simulation:

In [ ]:
function WolffXY(N::Integer, J::Real, T::Real, Nsw::Integer, bc::String, f=nothing)
    grid = Grid(N, J, T, bc)

    m = zeros(Nsw)
    m[1] = magnetizationXY(grid)
    if f != nothing
        @assert typeof(f)<:Function "Invalid function in WolffXY"
        farr = Array{Any}(undef, Nsw)
    end
    
    for i in 2:Nsw
        iterationWolff!(grid)
        m[i] = magnetizationXY(grid)
        if f != nothing
            farr[i] = f(grid)
        end
    end

    if f != nothing
        return grid, m, farr
    else
        return grid, m
    end
end

Where `magnetizationXY()` is a function which calculates the magnitude of total magnetization in the grid:

In [ ]:
function magnetizationXY(grid::Grid)
    vecs = [cos.(grid.grid), sin.(grid.grid)]
    m = norm(mean.(vecs))
    return m
end

To check the equilibration time we just plot the magnetization over time for the required sizes and temperatures:

In [ ]:
# physical constants
Nlist = 16:8:32
Tlist = [0.1, 1.0, 1.5]
J = 1

# iteration count
Neq = 2^7
Nsw = 2^13 + 2*Neq

# multithreading
dimT = length(Tlist)
dimN = length(Nlist)
dimTotal = dimT * dimN

# log arrays
Marr = zeros(length(Nlist), length(Tlist), Nsw)
Sarr = Array{Any}(undef, length(Nlist), length(Tlist))

# loop over all required temperatures and sizes
@time Threads.@threads for ind in range(start=dimTotal, stop=1, length=dimTotal)
    # load data from lists
    ind = Int(ind)
    Tind = mod(ind-1, dimT) + 1
    
    Nind = Int((ind - Tind) / dimT+1)

    T = Tlist[Tind]
    N = Nlist[Nind]
    
    # simulate
    println("Simulating N=", N, " and T=", T, "...")
    grid, m = WolffXY(N, J, T, Nsw, "closed")
    println("Done N=", N, " and T=", T)

    # log output
    Marr[Nind, Tind, :] = m
    Sarr[Nind, Tind] = grid.grid
end

In [ ]:
plist = []
hlist = []
for (Nind, N) in enumerate(Nlist)
    plt = plot(ylabel=@sprintf("N=%d", N), xlabel=(Nind==length(Nlist) ? "Iteration" : ""))
    for (Tind, T) in enumerate(Tlist)
        h = showGrid(Sarr[Nind, Tind], T)
        plot!(plt, Marr[Nind, Tind, :], label=@sprintf("T=%.1f",T), xaxis=:log)
        push!(hlist, h)
    end
    push!(plist, plt)
end

magplot = plot(plist..., plot_title="Magnetization Magnitude", ylims=(0, 1), layout=(3,1))
mapplot = plot(hlist..., plot_title="Final States", size=(600, 650), layout=(3,3))
display(magplot)
display(mapplot)

We can see that in general the $T=0.1$ case results ina a more ordered structure, althought thermal fluctuations still can disrupt the order. The cases $T=1$ is disordered, and $T=1.5$ is more disordered still, but not by much. therefore I guess that $T_c$ is lower than $T=1$.

In order to estimate the autocorrelation time we will define the error function from the tutorial:

In [ ]:
function calculateError(m, Neq, Nsw, minSize)
    err = Float64[]
    Nsw2 = 2^Int(floor(log(2, Nsw-Neq))) # closest power of 2 smaller than number of sweeps after equilibration
    ml = m[end-Nsw2+1:end]
    while length(ml) > minSize
        push!(err, std(ml)/sqrt(length(ml)-1))
        ml = map(j -> mean(ml[2*j-1:2*j]), range(1,stop=div(length(ml),2)) )
    end
    return err
end

And we will use it to estimate the autocorrelation time:

In [ ]:
plist = []
tau = zeros(dimN, dimT)
for (Nind, N) in enumerate(Nlist)
    for (Tind, T) in enumerate(Tlist)
        err = calculateError(Marr[Nind, Tind, :], Neq, Nsw, 2^5)
        tau[Nind, Tind] = round(((err[end]/err[1])^2-1)/2)
        p = plot(err, marker=:circle, label=nothing, title=@sprintf("N=%d, T=%.1f, τ=%d", N, T, tau[Nind, Tind]))
        push!(plist, p)
    end
end
plot(plist..., size=(800,650), layout=(3,3), plot_title="Error for different bin sizes")

And we see that the autocorrelation time is shorter near the phase transition at $T=1$, and longer elsewhere, even on lower temperatures. This does not make sense. At least, we do see that th e autocorrelation time increases with the system's size, as expected.

### 3. Correlation Function

Anyways, now we arrived at the interesting part: let's calculate the spin-spin correlation $C(|i-j|)=⟨\cos (\theta_i - \theta_j)⟩$. We will do that along the horizontal direction and average over the different rows, and do that for several temperatures. 

To calculate the correlation we will define a function:

In [ ]:
function correlationXY(grid::Matrix, dx::Integer)
    rownum = size(grid,1)
    N = size(grid, 2)
    corr = zeros(N)
    for rowind in 1:rownum
        row = grid[rowind,:]
        corr = (rowind-1).*corr .+ [cos(row[i] - row[mod(i+dx-1,N)+1]) for i in 1:N]
        corr ./= rowind
    end
    return mean(corr)
end

Now, we simulate. Note that we can simulate for a lot less time since we know that the equilibration time is short:

In [ ]:
# physical constants
N = 32
Tlist = [0.1, 1.5]
repeats = 32
J = 1

# multithreading
dimT = length(Tlist)
dimR = repeats
dimTotal = dimT * dimR

# iteration count
Nsw = 2^11

# log arrays
corr = zeros(N, dimT, dimR)

# loop over all required temperatures and sizes
@time Threads.@threads for ind in 1:dimTotal # @time Threads.@threads 
    # load data from lists
    Tind = mod(ind-1, dimT) + 1
    Rind = Int((ind - Tind) / dimT+1)
    T = Tlist[Tind]
    
    # simulate
    grid, m = WolffXY(N, J, T, Nsw, "closed")
    corr[:, Tind, Rind] = [correlationXY(grid.grid, i) for i in 0:(N-1)]
end

In [ ]:
plist = [plot(abs.(mean(corr[3:12,i,:], dims=2)), label="", title=@sprintf("T=%.2f", Tlist[i]), yaxis=:log, xscale=(Tlist[i]<1 ? :log10 : :identity)) for i in 1:2]
plot(plist..., marker=:circle, xlabel="Spin Distance", ylabel="Correlation", layout=(2,1))

And we indeed see that for $T\ll1$ the correlation is linear in a log-log plot, while for $T\gg1$ the correlation is linear in a semilogy plot, as expected. In the in-between region the behaviour is more erratic, since we use a finite size system whihc results in a relation which is between power-law and exponential, i.e. not linear in either cases.

### 4. Spin Stiffness

Next, we want to calculate spin stiffness on various system sizes and temperatures. We start by creating a function which calculates it:

In [ ]:
function spinStiffness(grid::Grid)
    N = grid.N
    T = 1/grid.beta

    ρs = -1/(2*N^2) * (energy(grid) + 2/T * (spinCurrent2(grid, "x") + spinCurrent2(grid, "y")) )
    return ρs
end

The functions `energy()` and `spinCurrent2()` are functions which calculate $⟨E⟩$ and $⟨I^2_{x,y}⟩$ respectively. We start with $E$:

In [ ]:
function energy(grid::Grid)
    N = grid.N
    J = grid.J
    H = 0
    for i in 1:N
        for j in 1:N
            thetai = grid.grid[i,j]
            H += sum([-J * cos(thetai-grid.grid[s[1],s[2]]) for s in nearestNeighboors(i, j, grid, "all")])
        end
    end
    H /= 2 # fix double counting
    return H
end

And next the spin currents:

In [ ]:
function spinCurrent2(grid::Grid, dir::String)
    I = spinCurrent(grid, dir)
    return I^2
end

function spinCurrent(grid::Grid, dir::String)
    @assert dir in ["x", "y"] "Invalid spin current direction " * dir

    N = grid.N
    J = grid.J

    I = 0
    for i in 1:N
        for j in 1:N
            thetai = grid.grid[i,j]
            I += sum([sin(thetai - grid.grid[s[1], s[2]]) for s in nearestNeighboors(i, j, grid, dir)])
        end
    end
    I /= 2 # fix double counting
    
    return I
end

We will test this on several temperatures and system sizes as instructed:

In [ ]:
Nlist = [4,8,16]
Tlist = range(0.1, 2.0, 9)

# iteration count
Nsw = 2^11

# multithreading
dimT = length(Tlist)
dimN = length(Nlist)
dimTotal = dimT * dimN

# log arrays
Marr = zeros(length(Nlist), length(Tlist), Nsw)
Sarr = Array{Any}(undef, length(Nlist), length(Tlist))
ρarr = zeros(length(Nlist), length(Tlist))

# loop over all required temperatures and sizes
@time Threads.@threads for ind in range(start=dimTotal, stop=1, length=dimTotal)
    # load data from lists
    ind = Int(ind)
    Tind = mod(ind-1, dimT) + 1
    
    Nind = Int((ind - Tind) / dimT+1)

    T = Tlist[Tind]
    N = Nlist[Nind]
    
    # simulate
    grid, m, ρs = WolffXY(N, J, T, Nsw, "closed", spinStiffness)

    # log output
    Marr[Nind, Tind, :] = m
    Sarr[Nind, Tind] = grid.grid

    # calculate stiffness
    ρarr[Nind, Tind] = mean(ρs[Int(Nsw/2):end])
end

By testing we know that a linear line is a fine fit to the stiffness in order to get th eintercpt at $T=0$:

In [ ]:
@. linear_model(x, p) = p[1] + p[2]*x
fits = [curve_fit(linear_model, Tlist, ρarr[Nind, :], [1.0, -1.0]) for Nind in 1:length(Nlist)]

plist = [plot(Tlist, ρarr[Nind, :], marker=:circle, title=@sprintf("N=%d",Nlist[Nind]), label="Data") for Nind in 1:length(Nlist)]
[plot!(p, Tlist, linear_model(Tlist, fits[pind].param), marker=:none, label="Fit", ylabel=@sprintf("Intercept=%.2f", fits[pind].param[1])) for (pind, p) in enumerate(plist)]
plot(plist...)

Indeed the intercept at $T\to 0$ is $\rho \to 1$. The case for $T\to \infty $ is a bit harder to infer from this specific plot, but by getting to larger temperatures we can indeed observe saturation at $\rho\to 0$. We can also check out the slope of the fit:

In [ ]:
[f.param[2] for f in fits]

And as expected the slope becomes steaper as the system's size increase.

### 5. KT Transition Temperature

Lastly, we want to get $T_{\rm KT}$. We will do that uing the procedure described in the instructions. First, we simulate and calculate the spin stiffness for each system size and temperature required:

In [ ]:
Nlist = [4,6,8,10]
Tlist = range(0.8, stop=1.0, length=16)

# iteration count
Nsw = 2^12

# multithreading
dimT = length(Tlist)
dimN = length(Nlist)
dimTotal = dimT * dimN

# log arrays
ρarr = zeros(length(Nlist), length(Tlist))

# loop over all required temperatures and sizes
@time Threads.@threads for ind in range(start=dimTotal, stop=1, length=dimTotal)
    # load data from lists
    ind = Int(ind)
    Tind = mod(ind-1, dimT) + 1
    Nind = Int((ind - Tind) / dimT+1)

    T = Tlist[Tind]
    N = Nlist[Nind]

    # simulate
    grid, m, ρs = WolffXY(N, J, T, Nsw, "closed", spinStiffness)

    # calculate stiffness
    ρarr[Nind, Tind] = mean(ρs[2^10:end])
end

Now we try to fit $\rho_s (L)$ as given to us to the data:

In [ ]:
Carr = zeros(length(Tlist))
Δarr = zeros(length(Tlist))
plist = []
for (Tind, T) in enumerate(Tlist)
    @. stiffness_model(x, a) = 2/pi*T*(   1 + (1/2)/(log(x)+a[1])   )
    fit = curve_fit(stiffness_model, Nlist, ρarr[:, Tind], [1e0])
    Carr[Tind] = fit.param[1]
    Δarr[Tind] = sqrt(estimate_covar(fit)[1])
    p = plot(Nlist, ρarr[:, Tind], label="Data", title=@sprintf("T=%.2f", T))
    plot!(Nlist, stiffness_model(Nlist, fit.param), label="Fit")
    push!(plist, p)
    
    if ~fit.converged
        println("Fit did not ocnverge for T=", T)
    end
end
plot(plist..., size=(1000, 600))

Even tho the fits do not resemble the data at all, we can get a greate approximation of the critical temperature by looking at the normalized error:

In [ ]:
plot(Tlist, Δarr./Carr.^2, yaxis=:log, marker=:circle, title="Normalized Error Δ(T)")

We see that $T_{\rm KT} \approx 0.88$, which is indded what we should according to [Phys. Rev. B 37, 5986(R) (1988)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.37.5986).

**Note: I know that the fits should have actually describe the data, and that the error sould not have been normalized. But I can't for the life of me find what have I done wrong, so I just used what I could get - and the final results is somewhow fine either way.**